In [1]:
from prophet.serialize import model_to_json, model_from_json

import pandas as pd
import yfinance as yf

import seaborn as sns

import matplotlib.pyplot as plt

import pytz

#sns.cubehelix_palette(start=.5, rot=-.75, as_cmap=True)
sns.set(style="whitegrid", color_codes=True)

# Set color for all bar plots
sns.set_palette('pastel')


def convert_df_col_time_from_utc_to_tz(df, col='ds', from_tz='UTC', to_tz='US/Eastern'  ):
    """
    Convert the time in the column from UTC to the timezone
    :param df: dataframe
    :param col: column name
    :return: dataframe
    """
    eastern = pytz.timezone(to_tz)
    df[col] = pd.to_datetime(df[col])
    df[col] = df[col].dt.tz_localize(from_tz).dt.tz_convert(eastern)
    return df


In [2]:
# Get BTC data from Yahoo Finance with interval 1h, for the past one year
btc_yf_origin = yf.download(tickers='BTC-USD', period='2y', interval='1h')

# Reset index and have Datetime as a column
btc_yf_origin.reset_index(inplace=True)

btc_yf_origin['Datetime'] = pd.to_datetime(btc_yf_origin['Datetime'])

# Drop timezone info
btc_yf_origin['Datetime'] = btc_yf_origin['Datetime'].dt.tz_localize(None)

btc_yf_origin.head()


[*********************100%%**********************]  1 of 1 completed


,Datetime,Open,High,Low,Close,Adj Close,Volume
0,2022-08-07 01:00:00,22894.796875,22937.226562,22894.796875,22921.767578,22921.767578,0
1,2022-08-07 02:00:00,22923.601562,22991.818359,22920.060547,22991.818359,22991.818359,0
2,2022-08-07 03:00:00,22994.587891,23021.953125,22991.279297,23000.361328,23000.361328,0
3,2022-08-07 04:00:00,22999.648438,23019.000000,22963.537109,22968.437500,22968.437500,51468288
4,2022-08-07 05:00:00,22965.314453,22999.927734,22965.314453,22980.751953,22980.751953,0


In [3]:
backtest_df = pd.read_pickle('bt-v5-init20240701-epoch100-steph6-batch16-train180.pkl')


# Order by mystery_end descending, rmse ascending
backtest_df = backtest_df.sort_values(by=['mystery_end', 'mape'], ascending=[False, True])
backtest_df

,current_datetime,training_start,training_end,validation_size,mystery_start,mystery_end,prophet_predictions,y,yhat,mse,mae,mape,std_error,rmspe,pcg_same_dir
61,2024-07-01 12:00:00,2024-01-18 19:00:00,2024-07-15 23:00:00,382,2024-07-16 00:00:00,2024-07-20 23:00:00,ds prophet_yhat prophe...,"[64611.44, 64848.08, 64819.91, 64752.14, 64487...","[62366.84, 62676.47, 63500.996, 63536.86, 6347...",9.134785e+06,2476.096720,0.037725,1733.127176,4.571812,0.546218
74,2024-07-01 12:00:00,2024-01-22 01:00:00,2024-07-15 23:00:00,366,2024-07-16 00:00:00,2024-07-20 23:00:00,ds prophet_yhat prophe...,"[64611.44, 64848.08, 64819.91, 64752.14, 64487...","[60134.633, 60731.938, 61299.402, 61855.875, 6...",9.120159e+06,2544.372457,0.038746,1626.753612,4.568856,0.546218
68,2024-07-01 12:00:00,2024-01-20 13:00:00,2024-07-15 23:00:00,373,2024-07-16 00:00:00,2024-07-20 23:00:00,ds prophet_yhat prophe...,"[64611.44, 64848.08, 64819.91, 64752.14, 64487...","[61125.06, 61049.914, 61303.668, 61490.555, 61...",1.261932e+07,3368.521579,0.051527,1127.999037,5.414996,0.487395
77,2024-07-01 12:00:00,2024-01-22 19:00:00,2024-07-15 23:00:00,362,2024-07-16 00:00:00,2024-07-20 23:00:00,ds prophet_yhat prophe...,"[64611.44, 64848.08, 64819.91, 64752.14, 64487...","[61591.58, 61943.55, 61777.875, 61756.88, 6165...",1.468476e+07,3405.879815,0.051887,1756.343859,5.795604,0.521008
75,2024-07-01 12:00:00,2024-01-22 07:00:00,2024-07-15 23:00:00,365,2024-07-16 00:00:00,2024-07-20 23:00:00,ds prophet_yhat prophe...,"[64611.44, 64848.08, 64819.91, 64752.14, 64487...","[61084.96, 60994.367, 61645.84, 62154.918, 622...",1.423172e+07,3410.422133,0.052086,1612.681562,5.727856,0.487395
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2024-07-01 12:00:00,2024-01-04 13:00:00,2024-07-02 12:00:00,385,2024-07-02 13:00:00,2024-07-07 12:00:00,ds prophet_yhat prophe...,"[62626.92, 62035.43, 61948.08, 61920.89, 62147...","[60421.207, 60156.918, 60098.305, 60333.37, 60...",7.192016e+06,2445.433421,0.042604,1100.850503,4.722568,0.579832
3,2024-07-01 12:00:00,2024-01-04 07:00:00,2024-07-02 06:00:00,385,2024-07-02 07:00:00,2024-07-07 06:00:00,ds prophet_yhat prophe...,"[62629.99, 62670.01, 62650.01, 62715.98, 62832...","[61331.902, 60838.086, 60881.88, 61016.996, 61...",8.805537e+06,2678.736402,0.046627,1276.678638,5.243196,0.546218
2,2024-07-01 12:00:00,2024-01-04 01:00:00,2024-07-02 00:00:00,385,2024-07-02 01:00:00,2024-07-07 00:00:00,ds prophet_yhat prophe...,"[62926.0, 62999.56, 63138.67, 63038.01, 63024....","[54144.9, 54618.07, 54876.402, 55336.23, 55970...",1.338434e+07,3084.742315,0.053139,1966.901851,6.308537,0.436975
1,2024-07-01 12:00:00,2024-01-03 19:00:00,2024-07-01 18:00:00,385,2024-07-01 19:00:00,2024-07-06 18:00:00,ds prophet_yhat prophe...,"[63226.78, 63292.53, 63130.01, 62975.1, 62899....","[55514.453, 56019.92, 56303.73, 56442.676, 566...",1.087905e+07,2725.084953,0.046665,1858.213524,5.641574,0.478992


In [4]:
with open('prophet_model_61.json', 'r') as fin:
    m = model_from_json(fin.read())  # Load model

In [5]:
# Use the model to make a forecast
future = m.make_future_dataframe(periods=120, freq='H')

forecast = m.predict(future)

forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]



,ds,yhat,yhat_lower,yhat_upper
0,2024-01-18 13:00:00,42426.034901,39969.755630,44913.003505
1,2024-01-18 14:00:00,42386.942102,39749.017110,44833.831428
2,2024-01-18 15:00:00,42357.119415,39726.946135,44789.311347
3,2024-01-18 16:00:00,42329.984963,39701.298619,44862.614838
4,2024-01-18 17:00:00,42293.535106,39667.927272,44815.555528
...,...,...,...,...
4422,2024-07-20 19:00:00,54767.869189,44509.420649,62720.682122
4423,2024-07-20 20:00:00,54753.048006,44403.734845,62800.303671
4424,2024-07-20 21:00:00,54762.403408,44907.350535,62388.838699
4425,2024-07-20 22:00:00,54780.159597,43749.277784,63079.027395


In [6]:
# Print range of forecast['ds']
print(forecast['ds'].min())
print(forecast['ds'].max())

2024-01-18 13:00:00
2024-07-20 23:00:00


In [7]:
# calculate rmse and mape of the forecast
test_y = backtest_df['y'][0]

#test_y

# Get the last 120 date in forecast
last_120 = forecast.tail(120)
last_120



,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,daily,daily_lower,daily_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
4307,2024-07-16 00:00:00,56438.150333,54412.565393,59734.279396,56438.150333,56438.150333,0.000372,0.000372,0.000372,0.009059,0.009059,0.009059,0.008687,0.008687,0.008687,0.0,0.0,0.0,56949.417354
4308,2024-07-16 01:00:00,56424.752369,54415.605896,59441.656569,56424.752369,56424.752369,-0.000417,-0.000417,-0.000417,0.007804,0.007804,0.007804,0.008221,0.008221,0.008221,0.0,0.0,0.0,56865.083817
4309,2024-07-16 02:00:00,56411.354405,54095.453009,59458.284653,56411.354405,56411.354405,-0.001093,-0.001093,-0.001093,0.006616,0.006616,0.006616,0.007710,0.007710,0.007710,0.0,0.0,0.0,56784.591837
4310,2024-07-16 03:00:00,56397.956441,54153.218203,59439.483467,56397.956441,56397.956441,-0.001261,-0.001261,-0.001261,0.005895,0.005895,0.005895,0.007156,0.007156,0.007156,0.0,0.0,0.0,56730.439226
4311,2024-07-16 04:00:00,56384.558478,54166.577829,59115.159862,56384.558478,56384.558478,-0.000939,-0.000939,-0.000939,0.005626,0.005626,0.005626,0.006565,0.006565,0.006565,0.0,0.0,0.0,56701.775584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4422,2024-07-20 19:00:00,54897.384493,44509.420649,62720.682122,45245.686938,62351.331690,-0.000330,-0.000330,-0.000330,-0.002359,-0.002359,-0.002359,-0.002029,-0.002029,-0.002029,0.0,0.0,0.0,54767.869189
4423,2024-07-20 20:00:00,54883.986529,44403.734845,62800.303671,45052.283471,62506.981325,-0.000385,-0.000385,-0.000385,-0.002386,-0.002386,-0.002386,-0.002000,-0.002000,-0.002000,0.0,0.0,0.0,54753.048006
4424,2024-07-20 21:00:00,54870.588565,44907.350535,62388.838699,44858.880003,62717.582936,0.000006,0.000006,0.000006,-0.001972,-0.001972,-0.001972,-0.001977,-0.001977,-0.001977,0.0,0.0,0.0,54762.403408
4425,2024-07-20 22:00:00,54857.190601,43749.277784,63079.027395,44665.476536,62931.681958,0.000552,0.000552,0.000552,-0.001404,-0.001404,-0.001404,-0.001957,-0.001957,-0.001957,0.0,0.0,0.0,54780.159597


In [8]:
# Calculate RMSE
rmse = ((last_120['yhat'] - test_y) ** 2).mean() ** .5

# Calculate MAPE
mape = (abs(last_120['yhat'] - test_y) / test_y).mean()



print(f'Prophet RMSE: {rmse}')
print(f'Prophet MAPE: {mape}')


Prophet RMSE: 4417.510256517579
Prophet MAPE: 0.06322341058625591
